In [11]:
import pandas as pd
import numpy as np
from nltk.tokenize import word_tokenize
from nltk import pos_tag
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.preprocessing import LabelEncoder
from collections import defaultdict
from nltk.corpus import wordnet as wn
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import model_selection, naive_bayes, svm
from sklearn.metrics import accuracy_score
import nltk
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [15]:
nltk.download('averaged_perceptron_tagger')
np.random.seed(500)

Corpus = pd.read_csv(r"train.csv",encoding='latin-1')

Corpus['tweet'].dropna(inplace=True)

Corpus['tweet'] = [entry.lower() for entry in Corpus['tweet']]

Corpus['tweet']= [word_tokenize(entry) for entry in Corpus['tweet']]
tag_map = defaultdict(lambda : wn.NOUN)
tag_map['J'] = wn.ADJ
tag_map['V'] = wn.VERB
tag_map['R'] = wn.ADV

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [17]:
nltk.download('stopwords')
for index,entry in enumerate(Corpus['tweet']):
    Final_words = []
    word_Lemmatized = WordNetLemmatizer()
    for word, tag in pos_tag(entry):
        if word not in stopwords.words('english') and word.isalpha():
            word_Final = word_Lemmatized.lemmatize(word,tag_map[tag[0]])
            Final_words.append(word_Final)
    Corpus.loc[index,'text_final'] = str(Final_words)

In [18]:
Train_X, Test_X, Train_Y, Test_Y = model_selection.train_test_split(Corpus['text_final'],Corpus['label'],test_size=0.3)

In [19]:
Encoder = LabelEncoder()
Train_Y = Encoder.fit_transform(Train_Y)
Test_Y = Encoder.fit_transform(Test_Y)

In [20]:
Tfidf_vect = TfidfVectorizer(max_features=5000)
Tfidf_vect.fit(Corpus['text_final'])
Train_X_Tfidf = Tfidf_vect.transform(Train_X)
Test_X_Tfidf = Tfidf_vect.transform(Test_X)

print(Tfidf_vect.vocabulary_)

{'user': 4687, 'father': 1568, 'selfish': 3905, 'drag': 1299, 'kid': 2414, 'run': 3797, 'thanks': 4450, 'lyft': 2656, 'credit': 1004, 'ca': 644, 'use': 4684, 'cause': 704, 'offer': 3085, 'van': 4698, 'bihday': 448, 'majesty': 2674, 'model': 2853, 'love': 2619, 'take': 4375, 'time': 4501, 'factsguide': 1530, 'society': 4104, 'motivation': 2888, 'huge': 2124, 'fan': 1547, 'big': 442, 'talk': 4379, 'leave': 2499, 'chaos': 736, 'pay': 3239, 'get': 1803, 'camp': 657, 'tomorrow': 4522, 'next': 3004, 'school': 3855, 'year': 4966, 'think': 4464, 'exams': 1484, 'hate': 1991, 'imagine': 2179, 'actorslife': 39, 'girl': 1819, 'land': 2456, 'cavs': 705, 'champion': 731, 'cleveland': 807, 'welcome': 4833, 'ireland': 2299, 'consumer': 933, 'price': 3456, 'index': 2205, 'mom': 2857, 'climb': 812, 'previous': 3454, 'may': 2741, 'blog': 489, 'silver': 4014, 'gold': 1847, 'forex': 1689, 'orlando': 3147, 'pulseshooting': 3525, 'orlandoshooting': 3151, 'heabreaking': 2012, 'value': 4697, 'see': 3893, 'dadd

In [21]:
SVM = svm.SVC(C=1.0, kernel='linear', degree=3, gamma='auto')
SVM.fit(Train_X_Tfidf,Train_Y)
# predict the labels on validation dataset
predictions_SVM = SVM.predict(Test_X_Tfidf)
# Use accuracy_score function to get the accuracy
print("SVM Accuracy Score -> ",accuracy_score(predictions_SVM, Test_Y)*100)

SVM Accuracy Score ->  96.0892689540098
